In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch

import warnings
warnings.filterwarnings("ignore") 

import sys
sys.path.append('../src')

In [3]:
# Names of the columns that are discrete
discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]

In [4]:
df = pd.read_csv('../data/adult/adult_train.csv')

In [5]:
df.shape

(26113, 15)

In [6]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [7]:
from synthesizers.metgan import *
metgan = MeTGANSynthesizer(embedding_dim=64,generator_dim=(256,256),discriminator_dim=(256,256),batch_size=500,discriminator_steps=5,verbose=True)

In [8]:
metgan.fit(df, discrete_columns, epochs=3)  # Epochs = 300 used in the actual experiments

Data_dim_p:  1570
Epoch 1, Loss G: -0.1847,Loss D: -2.4443
Epoch 2, Loss G: -0.2058,Loss D: -0.3489
Epoch 3, Loss G:  0.1077,Loss D: -0.1875


In [9]:
metgan.save(r'../models/model-adult.pth')

In [10]:
syn_data = metgan.sample(32561)  # Size of complete Adult dataset

In [11]:
syn_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,18,Private,159461,Some-college,12,Married-civ-spouse,Machine-op-inspct,Unmarried,White,Male,36,-3,50,United-States,<=50K
1,15,Private,22204,HS-grad,8,Married-civ-spouse,Prof-specialty,Unmarried,White,Male,14061,2,40,United-States,<=50K
2,32,Self-emp-not-inc,48230,Some-college,9,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,0,2087,58,United-States,>50K
3,44,Private,110546,Bachelors,8,Married-spouse-absent,Sales,Husband,Black,Male,75,-1,39,United-States,>50K
4,35,Private,341420,Bachelors,9,Married-civ-spouse,Other-service,Own-child,White,Male,48,7,55,United-States,>50K


In [12]:
syn_data.to_csv('../syn/metgan-adult-3epochs.csv',index=False)